In [189]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
import re
from IPython.display import clear_output

------
<a id="indice"></a>

# Índice

1. **[Coleta](#coleta)**
    * [Twitter Archive](#coleta:twitter-archive)
    * [Image Prediction](#coleta:image-prediction)
    * [Twitter API](#coleta:twitter-api)
    
    
2. **[Avaliação](#avaliacao)**
    * [Twitter Archive](#avaliacao:twitter-archive)
    * [Image Prediction](#avaliacao:image-prediction)
    * [Twitter API](#avaliacao:twitter-api)
    * [Notas da avaliação](#avaliacao:anotacoes)
    
    
3. **[Limpeza](#limpeza)**
    * [Definição](#limpeza:definicao)
    * [Twitter Archive](#limpeza:twitter-archive)
    * [Image Prediction](#limpeza:image-prediction)
    * [Twitter API](#limpeza:twitter-api)


4. **[Armazenamento](#armazenamento)**

------
<a id="coleta"></a>

# Coleta

<a id="coleta:twitter-archive"></a>

## Coleta: Twitter archive

In [2]:
df_twitter_arc = pd.read_csv('data/twitter-archive-enhanced.csv')
df_twitter_arc.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
297,837110210464448512,NaN,NaN,2017-03-02 01:20:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Clark. He passed pupper training today...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/837110210...,13,10,Clark,None,None,pupper,None
246,845677943972139009,NaN,NaN,2017-03-25 16:45:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",C'mon guys. Please only send in dogs. We only ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/845677943...,12,10,None,None,None,None,None


<a id="coleta:image-prediction"></a>

## Coleta: Image prediction

In [3]:
df_prediction = None

r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

if r.status_code is 200:
    df_prediction = pd.read_csv(pd.compat.StringIO(r.text), sep='\t')    
else:    
    print('ERROR: Image prediction request returned {status_code} status code.'.format(status_code = r.status_code))

In [4]:
df_prediction.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1350,759793422261743616,https://pbs.twimg.com/media/CotUFZEWcAA2Pku.jpg,2,golden_retriever,0.985876,True,Labrador_retriever,0.001948,True,kuvasz,0.001752,True
1121,726887082820554753,https://pbs.twimg.com/media/ChZr8SdWIAAVQKt.jpg,1,soft-coated_wheaten_terrier,0.515919,True,Irish_terrier,0.162655,True,Chesapeake_Bay_retriever,0.125182,True


<a id="coleta:twitter-api"></a>

## Coleta: Twitter API

In [5]:
# Carregar configuração do Twitter App
with open('twitter_config.json', 'r', encoding='utf-8') as file:
    app_config = json.load(file)

In [6]:
# Atribuição da configurações a variaveis locais
api_key = app_config['api_key']
api_secret = app_config['api_secret']
access_token = app_config['access_token']
access_secret = app_config['access_secret']

In [7]:
# Autentificação ao Twitter App
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Buscar dados do histórico de Tweets na API

In [8]:
json_path = 'data/tweet_json.txt'
error_log = 'data/tweet_error.log'

# Salvar Tweets localmente
if(not os.path.isfile(json_path)):
    print('Aguarde. Dado o volume de requisições, a coleta de tweets pode demorar alguns minutos.')
    tweet_count = 0
    for tweet_id in df_twitter_arc.tweet_id:
        print('{percent}%'.format(percent=int((++tweet_count/df_twitter_arc.tweet_id.size)*100)))
        try:
            status = api.get_status(tweet_id)
            with open(json_path, 'a', newline='\n') as file:
                file.write(f'{json.dumps(status._json)}\n')

        except Exception as err:
            with open(error_log, 'a', newline='\n') as log:
                log.write(f'{str(tweet_id)}: {str(err.args[0])}\n')
            print(f'{str(tweet_id)}: {str(err.args[0])}')
        
        clear_output(wait=True)
else:
    print('Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.')
        

Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.


In [9]:
# Criar uma lista de Dictionaries com os Tweets carregados
tweets = []

with open(json_path, 'r') as file:
    for line in file:
        try:
            tweet = json.loads(line)
            
            if(tweet.get('entities', False)):
                if(tweet['entities'].get('media', False) and tweet['entities']['media'][0].get('media_url', False)):
                    tweets.append({
                        'id': tweet['id'],
                        'created_at': tweet['created_at'],
                        'in_reply_to_status_id': tweet['in_reply_to_status_id'],
                        'in_reply_to_user_id': tweet['in_reply_to_user_id'],
                        'is_quote_status': tweet['is_quote_status'],
                        'retweet_count': tweet['retweet_count'],
                        'favorite_count': tweet['favorite_count'],
                        'media_url': tweet['entities']['media'][0]['media_url'],
                        'retweeted': tweet['retweeted'],
                        'favorited': tweet['favorited']
                    })
            
            
        except Exception as e:
            print(e) 
            
        

In [10]:
# Criar dataframe de Tweets consultados na API
columns = tweets[0].keys()
df_tweets_api = pd.DataFrame(tweets, columns = columns)

------
<a id="avaliacao"></a>

# Avaliação

<a id="avaliacao:twitter-archive"></a>

## Avaliação: Twitter data archive

In [11]:
 df_twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
195,855138241867124737,NaN,NaN,2017-04-20 19:16:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @frasercampbell_: oh my... what's that... b...,8.551225e+17,7.475543e+17,2017-04-20 18:14:33 +0000,https://twitter.com/frasercampbell_/status/855...,14,10,None,None,None,None,None
747,779834332596887552,NaN,NaN,2016-09-25 00:06:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Scout. He really wants to kiss himself...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/779834332...,11,10,Scout,None,None,None,None
122,869227993411051520,NaN,NaN,2017-05-29 16:24:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gizmo. His favorite thing is standing ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/869227993...,13,10,Gizmo,None,None,None,None
1830,676219687039057920,NaN,NaN,2015-12-14 01:58:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kenneth. He's stuck in a bubble. 10/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676219687...,10,10,Kenneth,None,None,None,None
916,756998049151549440,NaN,NaN,2016-07-23 23:42:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Oliver. He's an English Creamschnitzel...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/756998049...,11,10,Oliver,None,None,None,None


In [12]:
df_twitter_arc.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [13]:
df_twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id="twitter-arc:info"></a>
* Dados incompletos, contém apenas 2356 registros dos 5000 anunciados
* `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id` em `float`
* `timestamp` e `retweeted_status_timestamp` em `string`

In [14]:
# Verificar se há IDs duplicados
df_twitter_arc[df_twitter_arc.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


<a id="twitter-arc:source"></a>

In [15]:
# Visualizar exemplos de dados da coluna `source`
df_twitter_arc.loc[:, 'source'].sample(10)

120     <a href="http://twitter.com/download/iphone" r...
1320    <a href="http://twitter.com/download/iphone" r...
1600    <a href="http://twitter.com/download/iphone" r...
1538    <a href="http://twitter.com/download/iphone" r...
2034    <a href="http://twitter.com/download/iphone" r...
660     <a href="http://twitter.com/download/iphone" r...
1848    <a href="http://twitter.com/download/iphone" r...
1568    <a href="http://twitter.com/download/iphone" r...
1348    <a href="http://twitter.com/download/iphone" r...
832     <a href="http://twitter.com/download/iphone" r...
Name: source, dtype: object

Variável `source` é uma URL para a Aplicação utilizada no tweet, sem valor para a unidade de observação

<a id="twitter-arc:name"></a>

In [16]:
# Visualizar exemplos da coluna `name`
df_twitter_arc.name.sample(10)

447     Sunny
1212    Oscar
535      Cali
603      None
508      None
891      Emma
1789     None
658      None
298      None
946      None
Name: name, dtype: object

In [17]:
df_twitter_arc[df_twitter_arc.name == 'None'].name.count()

745

* Nomes (`name`) preenchidos com valor literal igual a `'None'`, supostamente um valor não preenchido, vazio

<a id="twitter-arc:category"></id>

In [18]:
# Visualizar exemplo de dados das colunas 'doggo' a 'puppo'
df_twitter_arc.loc[:,'doggo':].sample(10)

,doggo,floofer,pupper,puppo
772,None,None,pupper,None
646,None,None,None,None
979,None,None,None,None
949,None,None,None,None
1626,None,None,None,None
1767,None,None,None,None
2026,None,None,None,None
710,None,None,None,None
1852,None,None,None,None
1474,None,None,None,None


* Valores não preenchidos como 'None' literal

Uma variável em várias colunas

In [19]:
# Verificar padrão no preenchimento das classificações
df_twitter_arc.loc[:,'doggo':].nunique()

doggo      2
floofer    2
pupper     2
puppo      2
dtype: int64

In [20]:
df_twitter_arc.loc[:, 'doggo':'puppo'].isna().sum()

doggo      0
floofer    0
pupper     0
puppo      0
dtype: int64

<a id="twitter-arc:expanded_urls"></a>

Exemplos da coluna `expanded_urls`

In [21]:
df_twitter_arc.loc[:, 'expanded_urls'].sample(20)

1839    https://twitter.com/dog_rates/status/675891555...
878     https://twitter.com/dog_rates/status/760656994...
444     http://us.blastingnews.com/news/2017/01/200-do...
1722    https://twitter.com/dog_rates/status/680115823...
1048    https://twitter.com/dog_rates/status/743253157...
1298    https://twitter.com/dog_rates/status/707741517...
739     https://twitter.com/dog_rates/status/780601303...
777     https://twitter.com/dog_rates/status/776088319...
451     https://twitter.com/dog_rates/status/818627210...
591     https://twitter.com/dog_rates/status/799063482...
577     https://twitter.com/littlewiewel/status/800852...
1164    https://twitter.com/dog_rates/status/723179728...
2101    https://twitter.com/dog_rates/status/670691627...
278     https://twitter.com/dog_rates/status/840268004...
1224    https://twitter.com/dog_rates/status/714214115...
1481    https://twitter.com/dog_rates/status/693280720...
1144    https://twitter.com/dog_rates/status/727286334...
703     https:

In [22]:
df_twitter_arc.loc[:, 'expanded_urls'].isna().value_counts()

False    2297
True       59
Name: expanded_urls, dtype: int64

* Registros de `expanded_urls` com URLs para os twittes originais. Não apresenta valor a unidade em observacão

------
<a id="avaliacao:image-prediction"></a>

## Avaliação: Image prediction

In [23]:
df_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


<a id="image-prediction:info"></info>

In [24]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [25]:
df_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
809,692158366030913536,https://pbs.twimg.com/media/CZsKVxfWQAAXy2u.jpg,1,pug,0.956565,True,swing,0.018907,False,toy_poodle,0.013544,True
966,706291001778950144,https://pbs.twimg.com/media/Cc0_2tXXEAA2iTY.jpg,1,Border_terrier,0.587101,True,bull_mastiff,0.164087,True,Staffordshire_bullterrier,0.105011,True
993,708119489313951744,https://pbs.twimg.com/media/CdO-6x5W8AENSBJ.jpg,1,Norwich_terrier,0.264483,True,Norfolk_terrier,0.258786,True,chow,0.096899,True
358,672614745925664768,https://pbs.twimg.com/media/CVWbitUW4AAzclx.jpg,1,starfish,0.712717,False,goldfish,0.258865,False,sea_cucumber,0.002015,False
726,686358356425093120,https://pbs.twimg.com/media/CYZvRttWYAE_RXc.jpg,1,pug,0.985237,True,bull_mastiff,0.008841,True,boxer,0.002321,True
904,700143752053182464,https://pbs.twimg.com/media/CbdpBmLUYAY9SgQ.jpg,1,golden_retriever,0.532460,True,crossword_puzzle,0.103796,False,binder,0.100371,False
1080,718234618122661888,https://pbs.twimg.com/media/CfeukpmW4AEGjOE.jpg,1,malamute,0.370152,True,Siberian_husky,0.356398,True,Eskimo_dog,0.271042,True
1670,813081950185472002,https://pbs.twimg.com/media/C0ilsa1XUAEHK_k.jpg,2,Doberman,0.909951,True,kelpie,0.042649,True,miniature_pinscher,0.023004,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
496,675781562965868544,https://pbs.twimg.com/media/CWDbv2yU4AARfeH.jpg,1,Maltese_dog,0.921968,True,West_Highland_white_terrier,0.017811,True,toy_poodle,0.013555,True


<a id="image-prediction:p-values"></a>

Valores não padronizados para as variáveis `p1`, `p2` e `p3`

In [26]:
# Visualização de exemplos de valores nas colunas `p1`, `p2` e `p3`
df_prediction.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
263,brown_bear,toy_poodle,miniature_poodle
1543,Pembroke,Cardigan,basenji
728,Yorkshire_terrier,silky_terrier,Australian_terrier
1934,Labrador_retriever,golden_retriever,kuvasz
2019,golden_retriever,Labrador_retriever,cocker_spaniel


<a id="image-prediction:duplicated-urls"></a>

Verificar a duplicidade de Tweet ID (`tweet_id`) e Imagens (`jpg_url`)

In [27]:
# Verificar a duplicidade de IDs
df_prediction.tweet_id.nunique()

2075

Verificar a duplicidade na coluna `jpg_url`

In [28]:
df_prediction.jpg_url.duplicated().value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

Há 66 imagens duplicadas

In [29]:
# Validar indicadores de confiabiliade da predição
df_prediction.loc[:, ['p1_conf', 'p2_conf', 'p3_conf']].max()

p1_conf    1.000000
p2_conf    0.488014
p3_conf    0.273419
dtype: float64

------
<a id="avaliacao:twitter-api"></a>

## Avaliação: Twitter API Requests

In [30]:
df_tweets_api.describe()

,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,favorite_count
count,1.820000e+03,2.200000e+01,2.200000e+01,1820.000000,1820.000000
mean,7.237270e+17,6.992047e+17,4.196984e+09,2512.144505,6820.872527
std,5.777841e+16,4.409222e+16,0.000000e+00,4891.076169,11900.984651
min,6.660209e+17,6.671522e+17,4.196984e+09,11.000000,0.000000
25%,6.747671e+17,6.747625e+17,4.196984e+09,536.000000,1363.500000
50%,7.008223e+17,6.799651e+17,4.196984e+09,1118.000000,3153.000000
75%,7.617466e+17,7.032024e+17,4.196984e+09,2526.250000,7463.250000
max,8.924206e+17,8.558181e+17,4.196984e+09,82905.000000,163034.000000


<a id="tweets-api:info"></a>

In [31]:
df_tweets_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 10 columns):
id                       1820 non-null int64
created_at               1820 non-null object
in_reply_to_status_id    22 non-null float64
in_reply_to_user_id      22 non-null float64
is_quote_status          1820 non-null bool
retweet_count            1820 non-null int64
favorite_count           1820 non-null int64
media_url                1820 non-null object
retweeted                1820 non-null bool
favorited                1820 non-null bool
dtypes: bool(3), float64(2), int64(3), object(2)
memory usage: 104.9+ KB


In [32]:
df_tweets_api.sample(5)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
5,888917238123831296,Sun Jul 23 00:22:39 +0000 2017,NaN,NaN,False,4366,28414,http://pbs.twimg.com/media/DFYRgsOUQAARGhO.jpg,False,False
202,810284430598270976,Sun Dec 18 00:43:57 +0000 2016,NaN,NaN,False,12378,37556,http://pbs.twimg.com/media/Cz61ZD4W8AAcJEU.jpg,False,False
1800,666273097616637952,Mon Nov 16 15:14:19 +0000 2015,NaN,NaN,False,76,170,http://pbs.twimg.com/media/CT8T1mtUwAA3aqm.jpg,False,False
1767,666996132027977728,Wed Nov 18 15:07:24 +0000 2015,NaN,NaN,False,94,244,http://pbs.twimg.com/media/CUGlb6iUwAITEbW.jpg,False,False
296,790581949425475584,Mon Oct 24 15:53:19 +0000 2016,NaN,NaN,False,7818,21980,http://pbs.twimg.com/media/Cvi2FiIWAAAoDHc.jpg,False,False


* `created_at` como `string`
* `in_reply_to_status_id` e `in_reply_to_userid` como `Float`

<a id="tweets-api:duplicated"></a>
Verificar a duplicidade de valores

In [33]:
# Verificar se há IDs duplicados
df_tweets_api.id.duplicated().value_counts()

False    1820
Name: id, dtype: int64

In [34]:
# Verificar se há imagens duplicadas
df_tweets_api.media_url.duplicated().value_counts()

False    1759
True       61
Name: media_url, dtype: int64

Há tweets referenciando a mesma imagem

------

<a id="avaliacao:anotacoes"></a>

## Notas da avaliação
### Qualidade

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, contém apenas **2356** registros dos **5000** anunciados](#twitter-arc:info)
* [Coluna `source` sem valor à unidade de obsevação](#twitter-arc:source)
* [Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio](#twitter-arc:name)
* [Coluna `expanded_urls` sem valor à unidade de observação](#twitter-arc:expanded_urls)
* [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#image-prediction:p-values)
* [Duplicidade na coluna `jgp_url`](#image-prediction:duplicated-urls)


#### `df_tweets_api` - Twitter API
* [Diferentes Tweets referenciando a mesma imagem, duplicidade](#tweets-api:duplicated)


### Organização

#### `df_twitter_arc` - Twitter archive
1. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#twitter-arc:info)
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string`](#twitter-arc:info)
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Colunas como variáveis na identificação das predições](#image-prediction:info)


#### `df_tweets_api` - Twitter API
1. [Data de criação (`created_at`) como `string`](#tweets-api:info)
* [Valor do id na coluna `in_reply_to_status_id` como `float`](#tweets-api:info)
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#tweets-api:info)

------

<a id="limpeza"></a>

# Limpeza

<a id="limpeza:definicao"></a>

## Limpeza: Definição 

#### `df_twitter_arc` - Twitter archive
1. Base incompleta, de **5000**, apenas **2356** estão disponíveis
    * OK - Nada a ser feito
* [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
    * O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida
* [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`
* [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)(#twitter-arc:expanded_urls)
    * Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`.
* [Categorias \[doggo, floofer, pupper, puppo\] com literal `'None'`](#limpeza:twitter-arc:category)
    * Aplicar valor `np.nan` para nomes iguais a 'None'   
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
    * A coluna, `retweeted_status_id` ou `retweeted_status_user_id`, podem ser convertidas para `Boolean` como auxílio a identificação de tweets originais
    * Após a criação de uma coluna indicando se o tweet é um retweet, as colunas de retweet, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
    * As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
    * Converter valores para `datetime`
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
    * Reduzir a variável para uma coluna, `category`


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
    * Converter os valores para 'lower case' e substituir '_' por espaços
* [Previsões repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
    * Descartar predições sobre imagens repetidas
* [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
    * Transpor as variáveis nas colunas para linhas


#### `df_tweets_api` - Twitter API
1. [Data de criação (`created_at`) como `string`](#tweets-api:info)
    * Converter datas em string para o formato `datetime`
* [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `float`](#tweets-api:info)
    * Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir 0 as que estiverem sem valor


<a id="limpeza:twitter-archive"></a>

## Limpeza: Twitter archive

In [35]:
# Criar cópia do dataframe para a limpeza
df_arch_clean = df_twitter_arc.copy()

<a id="limpeza:twitter-arc:source"></a>

#### 2. [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
* O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida

In [36]:
# Remoção da coluna `source`
df_arch_clean.drop('source', axis='columns', inplace=True)

#### Teste: Remoção coluna `source`

In [37]:
# Conferir que a coluna `source` foi removida
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:name"></a>

#### 3. [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`

In [38]:
# Contagem de nomes iguais a 'None'
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

745

In [39]:
# Função para substituição de valores literais 'None' por np.nan
def set_nan_at_none(value):
    if str(value).lower() == 'none':
        return np.nan
    else:
        return value

In [40]:
# Aplicar `np.nan` aos nomes iguais a 'None'
df_arch_clean.name = df_arch_clean.name.apply(set_nan_at_none)

#### Teste: Aplicação de `np.nan` aos nomes como 'None'

In [41]:
df_arch_clean.name.isnull().sum()

745

In [42]:
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

0

In [43]:
df_arch_clean.name.sample(10)

1331         NaN
1905         NaN
1974    Fletcher
1071     getting
1286         NaN
1617     Gerbald
1818         NaN
1737           a
738         Koda
718         Finn
Name: name, dtype: object

<a id="limpeza:twitter-arc:expanded_urls"></a>

#### 4. [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)
* Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`

In [44]:
df_arch_clean.drop('expanded_urls', axis='columns', inplace=True)

#### Teste: Remoção da coluna `expanded_urls`

In [45]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'rating_numerator', 'rating_denominator',
       'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:category"></a>

#### 5. [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#limpeza:twitter-arc:category)
* Aplicar valor `np.nan` para nomes iguais a 'None'   

In [46]:
# Trocar valores 'None' para np.nan nas colunas `doggo`, `floofer`, `pupper`, `puppo`
df_arch_clean.doggo = df_arch_clean.doggo.apply(set_nan_at_none)
df_arch_clean.doggo.isna().value_counts()

True     2259
False      97
Name: doggo, dtype: int64

In [47]:
df_arch_clean.floofer = df_arch_clean.floofer.apply(set_nan_at_none)
df_arch_clean.floofer.isna().value_counts()

True     2346
False      10
Name: floofer, dtype: int64

In [48]:
df_arch_clean.pupper = df_arch_clean.pupper.apply(set_nan_at_none)
df_arch_clean.pupper.isna().value_counts()

True     2099
False     257
Name: pupper, dtype: int64

In [49]:
df_arch_clean.puppo = df_arch_clean.puppo.apply(set_nan_at_none)
df_arch_clean.puppo.isna().value_counts()

True     2326
False      30
Name: puppo, dtype: int64

In [50]:
# Soma de valores preenchidos nas classificações
sum_categories = 0
sum_categories += df_arch_clean.doggo.notna().sum()
sum_categories += df_arch_clean.floofer.notna().sum()
sum_categories += df_arch_clean.pupper.notna().sum()
sum_categories += df_arch_clean.puppo.notna().sum()

sum_categories

394

Apenas 394 dos 2346 registros foram classificados

<a id="limpeza:twitter-arc:info"></a>

#### 6. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
* Linhas com algum valor para as colunas `retweeted_status_id` ou `retweeted_status_user_id` podem ser descartadas por indicarem retweet, e o objetivo é analisar somente tweets originais
* Após a remoção retweets, as colunas `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
* As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`

#### Identificação dos tweets originais
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

In [51]:
df_arch_clean.retweeted_status_id.size

2356

In [52]:
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True     2175
False     181
Name: retweeted_status_id, dtype: int64

Dos 2356 registros, 181 são retweets

In [53]:
# Remoção dos retweets
df_arch_clean.drop(df_arch_clean[df_arch_clean.retweeted_status_id.notna()].index, axis=0, inplace=True)

#### Teste: Remoção de Retweets

In [54]:
# Consultar a existência de colunas com retweet_status_preenchido
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True    2175
Name: retweeted_status_id, dtype: int64

Remoção das colunas referentes a retweets

In [55]:
df_arch_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis='columns', inplace=True)

#### Teste: Verificação da remoção das colunas referentes a retweets

In [56]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'rating_numerator', 'rating_denominator', 'name', 'doggo',
       'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:datetime"></a>

#### 7. [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
* Converter valores para `datetime`

Haja vista que a coluna `retweet_status_timestamp` foi removida no passo anterior, não será necessária a sua conversão

In [57]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null object
dtypes: object(1)
memory usage: 34.0+ KB


In [58]:
df_arch_clean.timestamp = pd.to_datetime(df_arch_clean.timestamp)

#### Teste: Veriificação da conversão de `string` para `datetime`

In [59]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 34.0 KB


<a id="limpeza:twitter-arc:category-merge"></a>

#### 8. [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
* Reduzir a variável para uma coluna, `category`

In [60]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
doggo                    87 non-null object
floofer                  10 non-null object
pupper                   234 non-null object
puppo                    25 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(6)
memory usage: 220.9+ KB


In [61]:
df_arch_clean.loc[:, 'doggo':].notna().sum()

doggo       87
floofer     10
pupper     234
puppo       25
dtype: int64

In [62]:
df_arch_clean.loc[:, 'doggo':].notna().sum().sum()

356

De **2175** registros, apenas **356** estão classificados

Criar uma lista das categorias para mesclar as colunas, considerando a coluna a esquerda como prioritaria sobre valores nulos a direita.

In [63]:
category = df_arch_clean.doggo
category = category.combine_first(df_arch_clean.floofer)
category = category.combine_first(df_arch_clean.pupper)
category = category.combine_first(df_arch_clean.puppo)

df_arch_clean['category'] = category

In [64]:
df_arch_clean.category.notna().sum()

344

Após a unificação das colunas de classificação, pode-se ver que resultou em apenas **344** registros, diferente dos **356** iniciais. O que indica que haviam registros com mais de uma classificação.

In [65]:
df_arch_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis='columns', inplace=True)

#### Teste: Verificar a criação da coluna `category` a remoção das colunas `doggo`, `floofer`, `pupper` e `puppo`. 

In [66]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
category                 344 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(3)
memory usage: 169.9+ KB


------
<a id="limpeza:image-prediction"></a>

## Limpeza: Image prediction 

In [129]:
# Cópia do dataframe para limpeza
df_predict_clean = df_prediction.copy()

<a id="limpeza:image-prediction:names"></a>

#### 1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
* Converter os valores para 'lower case' e substituir '_' por espaços

In [130]:
clean_prediction_name = lambda value: str(value).lower().replace('_', ' ')

In [131]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
950,pug,teddy,mitten
1294,chow,Pomeranian,Siamese_cat
990,agama,Gila_monster,lumbermill
410,Brabancon_griffon,Chihuahua,golden_retriever
1672,dingo,Pembroke,basenji


Aplicar a padronização dos nomes em cada uma das colunas de predição

In [132]:
df_predict_clean.p1 = df_predict_clean.p1.apply(clean_prediction_name)

In [133]:
df_predict_clean.p2 = df_predict_clean.p2.apply(clean_prediction_name)

In [134]:
df_predict_clean.p3 = df_predict_clean.p3.apply(clean_prediction_name)

#### Teste: Verificar padronização dos nomes das predições

In [135]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
1916,chihuahua,toy poodle,labrador retriever
1906,barbell,dumbbell,go-kart
1958,chesapeake bay retriever,labrador retriever,weimaraner
596,mailbox,cannon,cuirass
1241,labrador retriever,golden retriever,doormat


<a id="limpeza:image-prediction:duplicated-urls"></a>

#### 2. [Predições repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
* Descartar predições sobre imagens repetidas

In [126]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [136]:
df_predict_clean.loc[:, ['jpg_url', 'p1', 'p2', 'p3']].duplicated().value_counts()

False    2009
True       66
dtype: int64

Dos **2075** registros, **66** são sobre a mesma imagem. Entretanto, seguindo o objetivo de trabalhar com apenas Tweets originais, além de descartar duplicidade é interessante também descartar as refêrencias a Retweets. Para isso, será feita uma consulta a seleção de Tweets originais, que foi efetuada no passo [6 da limpeza do Twitter Archive](#limpeza:twitter-arc:info)

In [137]:
df_predict_clean = df_predict_clean.loc[df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id)]

#### Teste: Seleção de Tweets originais

In [140]:
df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id).value_counts()

True    1994
Name: tweet_id, dtype: int64

Apenas dos **2075**, **1994** são de Tweets originais

------

Dos Tweets originai ainda é necessário verificar a existência de duplicidade de predições.

In [143]:
df_predict_clean.loc[:, ['jpg_url']].duplicated().value_counts()

False    1994
dtype: int64

Nenhuma duplicidade foi encontrada após a remoção das referências de Retweets

In [144]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    1994 non-null int64
jpg_url     1994 non-null object
img_num     1994 non-null int64
p1          1994 non-null object
p1_conf     1994 non-null float64
p1_dog      1994 non-null bool
p2          1994 non-null object
p2_conf     1994 non-null float64
p2_dog      1994 non-null bool
p3          1994 non-null object
p3_conf     1994 non-null float64
p3_dog      1994 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 161.6+ KB


<a id="limpeza:image-prediction:info"></a>

#### 3. [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
* Transpor as variáveis nas colunas para linhas:
    1. `[p1, p2, p3]`, para `prediction`
    * `[p1_conf, p2_conf, p3_conf]`, para `confidence`
    * `[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`

In [222]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1632,805826884734976000,https://pbs.twimg.com/ext_tw_video_thumb/80582...,1,siberian husky,0.248926,True,american staffordshire terrier,0.098313,True,eskimo dog,0.080188,True
1905,852226086759018497,https://pbs.twimg.com/ext_tw_video_thumb/85222...,1,prison,0.352793,False,dishwasher,0.110723,False,file,0.094112,False
264,670797304698376195,https://pbs.twimg.com/media/CU8mlhoVAAAteS5.jpg,1,pembroke,0.472197,True,beagle,0.090938,True,german shepherd,0.064366,True
437,674416750885273600,https://pbs.twimg.com/media/CVwCdCFW4AUHY4D.jpg,1,chihuahua,0.287201,True,boston bull,0.250920,True,whippet,0.141012,True
1312,754482103782404096,https://pbs.twimg.com/ext_tw_video_thumb/75448...,1,tub,0.596796,False,bathtub,0.381098,False,shower curtain,0.017629,False


**1. `[p1, p2, p3]`, para `prediction`**

In [232]:
# Organizando as variáveis de tentativa de predição para uma coluna apenas
df_predic_try = pd.melt(df_predict_clean,
        id_vars=['tweet_id', 'jpg_url'],
        value_vars=['p1', 'p2', 'p3'],
        var_name=['prediction_try'],
        value_name='prediction'
       )

In [233]:
df_predic_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction
5549,799757965289017345,https://pbs.twimg.com/media/CxlPnoSUcAEXf1i.jpg,p3,shetland sheepdog
4998,709519240576036864,https://pbs.twimg.com/media/Cdi3-f7W8AUOm9T.jpg,p3,flat-coated retriever
1940,880465832366813184,https://pbs.twimg.com/media/DDgK-J4XUAIEV9W.jpg,p1,golden retriever
2449,674781762103414784,https://pbs.twimg.com/media/CV1ObvEWcAA7c6i.jpg,p2,hamster
5979,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,p3,kelpie


Conversão dos valores da tentativa para número inteiro

In [234]:
# Metodo para retirar o índice da tentativa dos nomes das colunas
get_prediction_index = lambda value: int(re.sub(r'[^\d]', '', value))

In [235]:
df_predic_try.prediction_try = df_predic_try.prediction_try.apply(get_prediction_index)

In [236]:
df_predic_try.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 4 columns):
tweet_id          5982 non-null int64
jpg_url           5982 non-null object
prediction_try    5982 non-null int64
prediction        5982 non-null object
dtypes: int64(2), object(2)
memory usage: 187.0+ KB


In [237]:
df_predic_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction
5734,833124694597443584,https://pbs.twimg.com/media/C4_ad1IUoAEspsk.jpg,3,shopping cart
4894,700505138482569216,https://pbs.twimg.com/media/Cbixs3vUUAAqHHN.jpg,3,great dane
5052,715733265223708672,https://pbs.twimg.com/media/Ce7LlUeUUAEQkQl.jpg,3,toy poodle
1605,810284430598270976,https://pbs.twimg.com/media/Cz61ZD4W8AAcJEU.jpg,1,malamute
1963,885984800019947520,https://pbs.twimg.com/media/DEumeWWV0AA-Z61.jpg,1,blenheim spaniel


**2. `[p1_conf, p2_conf, p3_conf]`, para `confidence`**

In [249]:
# Movendo o nível de confiança das predições para uma coluna apenas
df_predic_conf = pd.melt(df_predic_clean,
                         id_vars=['tweet_id', 'jpg_url'],
                         value_vars=['p1_conf', 'p2_conf', 'p3_conf'],
                         var_name=['prediction_try'],
                         value_name='confidence'
                        )

In [250]:
df_predic_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
83,667495797102141441,https://pbs.twimg.com/media/CUNr4-7UwAAg2lq.jpg,p1_conf,0.143957
1799,831670449226514432,https://pbs.twimg.com/media/C4qv3JUW8AADirb.jpg,p1_conf,0.624802
3440,761599872357261312,https://pbs.twimg.com/media/CpG_CrlWYAYyuP3.jpg,p2_conf,0.224269
5516,761672994376806400,https://pbs.twimg.com/ext_tw_video_thumb/76167...,p3_conf,0.111565
3066,708026248782585858,https://pbs.twimg.com/ext_tw_video_thumb/70802...,p2_conf,0.068979


In [252]:
# Atribuir apenas valores inteiros aos indicadores de tentativa
df_predic_conf.prediction_try = df_predic_conf.prediction_try.apply(get_prediction_index)

In [253]:
df_predic_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
3476,768855141948723200,https://pbs.twimg.com/media/CquFrCKWAAAr32m.jpg,2,0.058365
5789,807010152071229440,https://pbs.twimg.com/media/CzMTcZoXUAEKqEt.jpg,3,0.031887
5893,822859134160621569,https://pbs.twimg.com/media/C2tiAzGXgAIFdqi.jpg,3,0.047745
2878,691483041324204033,https://pbs.twimg.com/media/CZikKBIWYAA40Az.jpg,2,0.077420
5796,808501579447930884,https://pbs.twimg.com/media/Czhf4XtVQAAIqpd.jpg,3,0.093193


**3. `[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`**

In [ ]:
# Resultado 

------
<a id="limpeza:twitter-api"></a>

## Limpeza: Twitter API

------
<a id="armazenamento"></a>

# Armazenamento

# Relatórios

* Data wrangling efforts
* Analyses and visualizations